In [31]:
import praw

# read CLIENT_ID and SECRET_TOKEN from a file
with open('reddit_credentials.txt') as f:
    CLIENT_ID = f.readline().strip()
    SECRET_TOKEN = f.readline().strip()

reddit = praw.Reddit(
    client_id=CLIENT_ID,
    client_secret=SECRET_TOKEN,
    user_agent="DataCollector",
)

url = "https://www.reddit.com/r/politics/comments/jptq5n/megathread_joe_biden_projected_to_defeat/"
submission = reddit.submission(url=url)

In [32]:
import networkx as nx

G = nx.DiGraph()

submission.comments.replace_more(limit=None)
comment_queue = submission.comments[:]  # Seed with top-level
for comment in comment_queue:
    G.add_edge(submission.id, comment.id)
G.nodes[submission.id]["text"] = submission.selftext
while comment_queue:
    comment = comment_queue.pop(0)
    G.nodes[comment.id]["text"] = comment.body
    comment_queue.extend(comment.replies)
    for reply in comment.replies:
        G.add_edge(comment.id, reply.id)

In [43]:
import pickle
with open("reddit_graph.pkl", "wb") as file:
    pickle.dump(G, file)